# DPLL



In [ ]:
def choose_literal_dlis(clauses: list[list[int]]) -> int:
    """
    DLIS: Dynamic Largest Individual Sum.
    Kies de literal (met teken) die het vaakst voorkomt in de huidige formule.
    """
    counts: Dict[int, int] = {}

    for clause in clauses:
        for lit in clause:
            counts[lit] = counts.get(lit, 0) + 1

    if not counts:
        raise ValueError("No literals to choose from (empty clause set).")

    # Kies literal met hoogste count
    best_lit = max(counts, key=counts.get)
    return best_lit

def has_empty_clause(clauses: list[list[int]]) -> bool:
    return any(len(clause) == 0 for clause in clauses)


def dpll(clauses: list[list[int]],
         num_vars: int,
         assignment: Optional[list[int]] = None) -> Tuple[bool, list[int]]:
    """
    DPLL met:
      - simplify (unit + pure + tautologies)
      - DLIS splitting heuristic
    assignment: lijst van literals die al beslist zijn.
    """
    if assignment is None:
        assignment = []

    # 1. Eerst simplificatie uitvoeren
    simplified_clauses, implied = simplify(clauses)

    # 2. Assignment uitbreiden met implicaties van simplify
    full_assignment = assignment.copy()
    for lit in implied:
        # conflict check: x en -x allebei toegewezen?
        if -lit in full_assignment:
            return False, []
        if lit not in full_assignment:
            full_assignment.append(lit)

    # 3. Check op UNSAT: lege clausule aanwezig?
    if has_empty_clause(simplified_clauses):
        return False, []

    # 4. Check op SAT: geen clausules meer
    if not simplified_clauses:
        # alle clausules zijn voldaan
        return True, full_assignment

    # 5. DLIS: kies een literal om op te splitsen
    split_lit = choose_literal_dlis(simplified_clauses)

    # 6. Probeer eerst met split_lit = True
    sat, model = dpll(
        simplified_clauses + [[split_lit]],
        num_vars,
        full_assignment + [split_lit]
    )
    if sat:
        return True, model

    # 7. Anders probeer met split_lit = False (dus -split_lit)
    sat, model = dpll(
        simplified_clauses + [[-split_lit]],
        num_vars,
        full_assignment + [-split_lit]
    )
    return sat, model

In [ ]:
from typing import List, Tuple, Dict, Optional

Assignment = Dict[int, bool]

def delete_tautologies(clause: list[int], clauses: list[list[int]]) -> int:
    taut_count = 0
    for literal in clause:
        opposite_literal = literal * -1
        if opposite_literal in clause:
            # delete clause from clause list
            #clauses.remove(clause)
            taut_count += 1
            return taut_count
    return taut_count

def pure_literal(clause: list[int], clauses: list[list[int]]) -> Tuple[list[list[int]], list[int]]:
    # Nog veel werk aan de winkel
    truth_list = []

    for literal in clause:
        pure = True
        neg_lit = literal * -1
        # Check if only negative or positive in other clauses
        for loop_clause in clauses:
            # Finding prove of literal not being pure
            if neg_lit in loop_clause:
                pure = False
                break
        # Take literal out of all clauses if it is pure
        if pure:
            # print(f"literal {literal} is seen as pure")
            if not literal in truth_list:
                truth_list.append(literal)
            for loop_clause in clauses[:]:
                if literal in loop_clause:
                    clauses.remove(loop_clause)

    return clauses, truth_list



def unit_clause(clause: list[int], clauses: list[list[int]]) -> Tuple[list[list[int]], int, int]:
    if len(clause) == 1 and not clause[0] == 0:
        truth = clause[0]
        #clauses.remove(clause)
        """
        for clause_loop in clauses[:]:
            opposite = truth * -1
            if truth in clause_loop:
                clauses.remove(clause_loop)
            elif opposite in clause_loop:
                clause_loop.remove(opposite)        
        """
        new_clauses = []
        opposite = -truth

        for clause in clauses:
            if truth in clause:
                continue  # deze clause is voldaan, dus overslaan
            if opposite in clause:
                clause = [lit for lit in clause if lit != opposite]
            new_clauses.append(clause)

        clauses = new_clauses

        # Take clause out of all clauses
        """for clause_loop in clauses:
            if truth in clause_loop:
                clause_loop.remove(truth)
            elif truth * -1 in clause_loop:
                clause_loop.remove(truth * -1)
        """
        unit = 1
    else:
        unit = 0
        truth = 0
    return clauses, truth, unit



def simplify(clauses: list[list[int]]) -> list[list[int]]:
    # CNF moet gesolved worden:
    truth_list = list()
    # Step 1: simplify
    terminate = 0
    while not terminate: # Terminate when clauses is simplified
        taut_tot = 0
        pure_tot = 0
        unit_tot = 0
        # maybe a clauses copy?
        for clause in clauses[:]:

            # We are deleting from the clauses list so skipp a clause that has already been delete
            if not clause in clauses: continue

            # Tautology check
            taut = delete_tautologies(clause, clauses)
            taut_tot += taut
            if taut == 1:
                clauses.remove(clause)
                continue
            # Only continue in this clause if clause is not a tautology

            # Unit clause check
            clauses, truth, unit = unit_clause(clause, clauses)
            unit_tot += unit
            if not truth == 0: # Unit literal was found
                if not truth in truth_list:
                    truth_list.append(truth)
                    continue # Only pure literal check needed if no unit literal was found -> no double work

            # Pure Literal check
            clauses, truth = pure_literal(clause, clauses)
            pure_tot += len(truth)
            if not truth == list():
                for truth_item in truth_list:
                    if not truth_item in truth:
                        truth_list.append(truth_item)

        # Check if all are taken out of clauses
        # print(f"taut_tot: {taut_tot}, pure_tot: {pure_tot}, unit_tot: {unit_tot}")
        # print(f"truth_list: {truth_list} ")
        # print(f"and clauses: {clauses}")

        if taut_tot == 0 and pure_tot == 0 and unit_tot == 0: # ... because then nothing more to simplify
            terminate = 1


    return clauses, truth_list





def lit_var(lit: int) -> int:
    return abs(lit)


def literal_value(lit: int, assignment: Assignment) -> Optional[bool]:
    v = abs(lit)
    if v not in assignment:
        return None
    val = assignment[v]
    return val if lit > 0 else (not val)



def dlis_pick_literal(clauses: List[List[int]], assignment: Assignment) -> Optional[int]:
    score: Dict[int, int] = {}

    for clause in clauses:
        # skip satisfied clauses
        if any(literal_value(lit, assignment) is True for lit in clause):
            continue

        # for unsatisfied clauses, count unassigned literals
        for lit in clause:
            val = literal_value(lit, assignment)
            if val is None:  # unassigned
                score[lit] = score.get(lit, 0) + 1

    if not score:
        return None

    best_lit = max(score.items(), key=lambda kv: kv[1])[0]
    return best_lit


def first_unassigned_literal(clauses: List[List[int]], assignment: Assignment) -> Optional[int]:
    for clause in clauses:
        for lit in clause:
            v = lit_var(lit)
            if v not in assignment:
                return lit
    return None


def pick_literal(
    clauses: List[List[int]],
    assignment: Assignment,
    heuristic: str = "dlis"
) -> Optional[int]:
    if heuristic == "dlis":
        return dlis_pick_literal(clauses, assignment)
    elif heuristic == "first":
        return first_unassigned_literal(clauses, assignment)
    else:
        raise ValueError(f"Unknown heuristic: {heuristic}")


def dpll(
    clauses: List[List[int]],
    assignment: Optional[Assignment] = None,
    heuristic: str = "dlis"
) -> Tuple[bool, Optional[Assignment]]:
    """
    clauses: CNF as list of clauses (each clause is a list of ints)
    assignment: current partial assignment {var: bool}
    heuristic: "dlis" or "first"
    Returns (is_sat, model)
    """
    if assignment is None:
        assignment = {}

    # 1) Simplify with your procedure (tautologies, unit, pure literals, ...)
    clauses, implied_truths = simplify(clauses)

    # 2) Add implied truths to the assignment
    for lit in implied_truths:
        v = abs(lit)
        val = lit > 0
        # Optionally: check for inconsistency with existing assignment
        if v in assignment and assignment[v] != val:
            return False, None
        assignment[v] = val

    # 3) Check for empty clause -> UNSAT
    if any(len(c) == 0 for c in clauses):
        return False, None

    # 4) If no clauses left, SAT
    if not clauses:
        return True, assignment

    # 5) Choose splitting literal
    lit = pick_literal(clauses, assignment, heuristic)
    if lit is None:
        # all variables assigned, but we didn't satisfy all clauses -> UNSAT
        return False, None

    # 6) Branch: first try lit = True (add [lit] as a unit clause)
    clauses_true = clauses + [[lit]]
    sat, model = dpll(clauses_true, assignment.copy(), heuristic)
    if sat:
        return True, model

    # 7) If that fails, try lit = False (add [-lit])
    clauses_false = clauses + [[-lit]]
    return dpll(clauses_false, assignment.copy(), heuristic)



if __name__ == "__main__":
    cnf = [
        [1, -2],
        [-1, 3],
    ]

    is_sat, model = dpll(cnf, heuristic="dlis")  # or "first"
    if is_sat:
        print("SAT")
        print(model)
    else:
        print("UNSAT")
